In [1]:
from matplotlib import pyplot
from numpy import asarray
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import pickle
import cv2
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from keras.preprocessing.image import ImageDataGenerator
from sklearn.svm import LinearSVC
import tensorflow as tf
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from keras.preprocessing.image import img_to_array
from keras.models import load_model
import os

c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
liveness_model = load_model("liveness.model")
liveness_le = pickle.loads(open("le.pickle", "rb").read())

W1223 11:52:26.768497  1776 deprecation_wrapper.py:119] From c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W1223 11:52:27.520138  1776 deprecation.py:323] From c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\tensorflow\python\ops\math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1223 11:52:29.069718  1776 deprecation_wrapper.py:119] From c:\users\hp\appdata\local\programs\python\python36\lib\site-packages\keras\backend\tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



In [3]:
from align import AlignDlib
alignment = AlignDlib('models/landmarks.dat')
def align_image(img):
    return alignment.align(224, img, alignment.getLargestFaceBoundingBox(img), 
                           landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)

In [4]:
def get_embedding(face):
    face = [face]
    samples = asarray(face,'float32')
    samples = preprocess_input(samples, version=2)  
    interpreter = tf.lite.Interpreter(model_path="sparse_mnist.tflite")
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    input_data = np.array(samples, dtype=np.float32)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    yhat = interpreter.get_tensor(output_details[0]['index'])

    return yhat

In [5]:
def train(embedded,targets):
    
    encoder = LabelEncoder()
    encoder.fit(targets)
    # Numerical encoding of identities
    y = encoder.transform(targets)
    
    embedded = np.array(embedded)
    print(embedded)
    
    # 50 train examples of 10 identities (5 examples each)
    X_train = embedded
    # 50 test examples of 10 identities (5 examples each)

    y_train = y
    
    svc = LinearSVC()
#     SVC(kernel='linear', probability=True)
    svc.fit(X_train, y_train)


    model_dump ={
    "classifier" : svc,
    "embeddings" : embedded, 
    "targets" : targets
    } 
    with open('model_livenes.pkl', 'wb') as config_dictionary_file:
          pickle.dump(model_dump, config_dictionary_file)

In [6]:
def trainfirst(embedded,targets):
    
    print(embedded)
    clf = svm.OneClassSVM(kernel="rbf", gamma=0.000001)
    clf.fit(embedded)


    model_dump ={
    "classifier" : clf,
    "embeddings" : embedded, 
    "targets" : targets
    } 
    with open('model_livenes.pkl', 'wb') as config_dictionary_file:
          pickle.dump(model_dump, config_dictionary_file)


In [19]:
## without augmentation
def webcapture():
    webcam = cv2.VideoCapture(1)
    i=0
    PADDING = 25
    embedding = []
    
    while True:
        ret, img = webcam.read()
        i =i+1
        if i > 10:
            frame = img
            try:
                faces = align_image(img)
            except:
                print("please look at camera side")
            try:
    #             print(len(faces))
                cv2.imshow("Frame", frame)
                cv2.imshow("Frame1", faces)
                key = cv2.waitKey(1) & 0xFF

                # if the `q` key was pressed, break from the loop
                if key == ord("q"):
                    break
    #             print(len(faces))
                if (len(faces) != 0): 
                    face = cv2.resize(faces, (32, 32))
                    face = face.astype("float") / 255.0
                    face = img_to_array(face)
                    face = np.expand_dims(face, axis=0)
                    preds = liveness_model.predict(face)[0]
                    j = np.argmax(preds)
                    label = liveness_le.classes_[j]
                    print(label,preds[j])
                    if label == "real" and preds[j] > 0.85:
                        webcam.release()
                        cv2.destroyAllWindows()
                        embedding.append(get_embedding(faces))
                        return embedding
                    else:
                        print("fake face")
            except:
                pass

In [8]:
def enter_face():
    embeddings = []
    for i in range(0,10):
        print(f"{i+1} capturing")
        face = webcapture()
        embeddings.append(face)
    return embeddings

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
def predictfisrt():
    old_model = pickle.load(open("model_livenes.pkl", 'rb'))
    clfmodel = old_model["classifier"]
    target = old_model["targets"]
    embeddings = old_model["embeddings"]
    target = np.unique(target)
    embedding = webcapture()
    avg = 0
    for i in embeddings:
        avg += cosine_similarity(embedding[0], [i])
    avg = avg/10
    print("avg",avg)
    if clfmodel.decision_function(embedding[0]) > -0.004 or avg > 0.60 :
        print(f'Recognized as {target}');
    else :
        print('not known');

In [36]:
def predict():
    old_model = pickle.load(open("model_livenes.pkl", 'rb'))
    clfmodel = old_model["classifier"]
    targets = old_model["targets"]
    embeddings = old_model["embeddings"]
    target = np.unique(targets)
    embedding = webcapture()
    example_prediction = clfmodel.predict(embedding[0])
    print(target)
    print(clfmodel.decision_function(embedding[0]))
    for i in range(len(target)):
        print(targets[i*10])
        avg = 0
        for j in range(i*10,(i*10)+10):
            avg += cosine_similarity([embeddings[j]], embedding[0])
        avg = avg/10
        print("avg",avg)
    if len(target)  >2 :
        probablity = (1/(1+np.exp(-list(clfmodel.decision_function(embedding[0])[0])[np.argmax(clfmodel.decision_function(embedding[0]))])))
        print( "confidence",list(clfmodel.decision_function(embedding[0])[0])[np.argmax(clfmodel.decision_function(embedding[0]))])
        print("probability",probablity)
        if abs(list(clfmodel.decision_function(embedding[0])[0])[np.argmax(clfmodel.decision_function(embedding[0]))]) > 0.50  :
            print(f'Recognized as {target[example_prediction[0]]}');
        else :
            print('not known');
    else:
        print(abs(clfmodel.decision_function(embedding[0])))
        print(abs(clfmodel.decision_function(embedding[0])) > 0.50)
        if abs(clfmodel.decision_function(embedding[0])) > 0.50  :
            print(f'Recognized as {target[example_prediction[0]]}');
        else :
            print('not known');

In [41]:
while True:
    print("press 1 for enter face in database")
    print("press 2 for see magic")
    print("press 3 for exit")
    choice = int(input())
    if choice == 1:
        name = input("enter name of person")
        if os.path.exists('model_livenes.pkl'):
            old_model = pickle.load(open("model_livenes.pkl", 'rb'))
            target = old_model["targets"]
            embeddings = old_model["embeddings"]
            print(embeddings)
            if name.strip() in target:
                print("class is already present")
            else:
                embedded = enter_face()
                new_embedded =[]
                for i in embedded:
                    new_embedded.append(i[0][0])
                new_embedded = np.array(new_embedded)
                embeddings = np.concatenate((embeddings, new_embedded), axis=0)
                for i in range(0,10):
                    target.append(name)
                train(embeddings,target)
        else:
            target= []
            embeddings = enter_face()
            embedded =[]
            for i in embeddings:
                embedded.append(i[0][0])
            embedded = np.array(embedded)
            for i in range(0,10):
                    target.append(name)
            trainfirst(embedded,target)
    elif choice == 2:
        if os.path.exists('model_livenes.pkl'):
            old_model = pickle.load(open("model_livenes.pkl", 'rb'))
            target = np.unique(old_model["targets"])
            if len(target) > 1:
                predict()
            else : 
                predictfisrt()
        else:
            print("first do training")
        break
    else :
        break

press 1 for enter face in database
press 2 for see magic
press 3 for exit
2
fake 0.93117964
fake face
real 0.89629084
['het' 'kumar']
[0.87802659]
kumar
avg [[0.89001286]]
het
avg [[0.48434168]]
[0.87802659]
[ True]
Recognized as kumar
